## A very bsic illustration of hash-based commit/reveal

In [1]:
!pip install pycryptodome

In [97]:
# Note: "Crypto" here is pycryptodome
#from Crypto import Random
from Crypto.Random import random
#from Crypto.Util import number
from Crypto.Hash import keccak  # keccak256 is the Ethereum utility hash func

Assume Bob and Alice would like to play a round of "rock, paper, scissors"

In [199]:

rps_choices = ['rock', 'paper', 'scissors']

def rps_winner( choiceA, choiceB):
    if choiceA == choiceB:
        return 'tie'
    return choiceA if rps_choices.index(choiceA) == (rps_choices.index(choiceB)+1)%3 else choiceB   
    

In [200]:
def salted_hash(choice, salt):
    # both arg are stings
    keccak_hash = keccak.new(digest_bits=256)
    s = choice + salt
    return keccak_hash.update(bytes(s,'ascii')).hexdigest()
    

In [201]:
def get_salt(bits=256):
    return str(random.getrandbits(bits))

Since they are in separate locations, they can't just "count it off" on their hands.

So instead they both create a hash commitment and send it to the other. Once both have received commitments, they open them.

Any variables that begin with either 'A_' or 'B_' are visible to only Alice or Bob, respectively


In [231]:
# alice chooses
A_aliceChoice = rps_choices[random.randint(0,2)]
A_aliceSalt = get_salt()
A_aliceCommit = salted_hash(A_aliceChoice, A_aliceSalt)

A_aliceChoice, A_aliceSalt, A_aliceCommit

('scissors',
 '79179575517037189523196150405479390125586574776350358096179094522451375024544',
 '5e26bd93fef7383b5e07f7fb43da837dcc7fe034ab394e09149c1965dd814b98')

In [232]:
# Alice sends Bob her commitment
B_aliceCommit = A_aliceCommit

In [233]:
# Even though Bob knows Alice's commitment, he can't know her choice, so he chooses and commits

B_bobChoice = rps_choices[random.randint(0,2)]

B_bobChoice

'paper'

Now here's an interesting thing. There are 2 ways things can go at this point:

a) Bob can create a hash commitment and send it to Alice, at which point they both reveal (choice,salt) to one another 
and individually validate one another's commitments and determine who has won. 

b) Bob, knowing Alice can't cheat since he has her commitment, can skip the whole hash thing and simply send her his choice. Since his choice *is* a public commitment he cannot claiom it wasn anything else. Alice can just send him the result - with her salt, she _does_ have to prove her choice and they can just move along.

Option b) probably requires that the game be synchronized and everyone already know that Alice is going to do a hash commit and Bob will just reveal.

In an asynchronous situation it's probably best that they both create and send a commitment hash and reveal when they have gotten a commitment from the other. The messages might pass one another ion the wire, but it'll work.


In [234]:
# Here's b: Bob just reveals

A_bobChoice_b = B_bobChoice

In [235]:
# Alice finds the winner:

winningChoice = rps_winner(A_aliceChoice, A_bobChoice_b)

winningPlayer = 'none' if winningChoice == 'tie' else 'Alice' if winningChoice == A_aliceChoice else 'Bob'

B_aliceReveal = ( winningPlayer, A_aliceChoice, A_aliceSalt) # reveal + winning player

winningPlayer

'Alice'

In [236]:
# Bob needs to verify that Alice's commitment matches the choice value she has revealed

(B_aliceWinner, B_aliceChoice, B_aliceSalt) = B_aliceReveal

if B_aliceCommit != salted_hash(B_aliceChoice, B_aliceSalt):
    print("Alice has cheated!")
else:
    print("Alice was honest")       

winningChoice = rps_winner(B_aliceChoice, B_bobChoice)
    
winningPlayer = 'none' if winningChoice == 'tie' else 'Alice' if winningChoice == A_aliceChoice else 'Bob'

if winningPlayer != B_aliceWinner:
    print("Something has gone terribly wrong")  
    
print(winningPlayer)   



Alice was honest
Alice


In [230]:
# And since he already has her commitment, he not only sends shis, he goes ahead and reveals
A_bobCommit = b_BobCommit
A_bobReveal = ( B_bobChoice, B_bobSalt)

A_bobReveal

NameError: name 'b_BobCommit' is not defined

In [149]:
# On getting Bob's reveal alice

# So... who won?



B_aliceReveal = ( A_aliceChoice, A_aliceSalt)